In [5]:
library(qvalue)
library(dplyr)
library(ggplot2)

In [6]:
Giordanos_selection = "/hps/nobackup/stegle/users/galvari/data/iPSCs/singleCell/metadata/ensembl_gene/Ensembl_75_Gene_CV_quant5.txt"
G_file = read.csv(Giordanos_selection, sep = "\t")
nrow(G_file)
head(G_file,2)

[1] 21592

,gene,mean,std,CV,quartile_cv
2,ENSG00000227232,3.1805905,3.6232403,1.139172,2
3,ENSG00000243485,0.2003178,0.8649483,4.317881,3


In [7]:
df_common_genes = read.csv("/hps/nobackup/stegle/users/acuomo/all_scripts/sc_eqtl/common_genes_across_all_aggregation_methods_and_bulk.csv")
common_genes = as.character(df_common_genes$gene)
length(common_genes)

[1] 12720

In [1]:
bulk_matched_folder = "/hps/nobackup/hipsci/scratch/ComparingQtlMapping/Bulk/BulkDay0Healthy_PCA20_88_206/"

In [8]:
# res = read.csv(paste0(bulk_matched_folder,"qtl_results_all.txt"), sep = "\t")
nrow(res)

[1] 16691457

In [6]:
bulk_folder = "/hps/nobackup/hipsci/scratch/ComparingQtlMapping/Bulk/BulkTotal_PCA20/"

In [9]:
# res_all = read.csv(paste0(bulk_folder,"qtl_results_all.txt"), sep = "\t")
nrow(res_all)

[1] 18974762

In [2]:
leads = read.csv(paste0(bulk_matched_folder,"top_qtl_results_all.txt"), sep = "\t")
nrow(leads)
head(leads)

[1] 43103

snp_id,p_value,beta,beta_se,empirical_feature_p_value,feature_chromosome,feature_start,feature_end,n_samples,n_e_samples,alpha_param,beta_param,snp_chromosome,snp_position,assessed_allele,call_rate,maf,hwe_p,feature_id
19_54622488_C_T,8.729855e-03,-0.1991305,0.07593285,0.602212835,19,54641444,54659419,88,88,0.5684955,27.30333,19,54622488,T,1,0.4204545,0.38051445,ENSG00000088038
19_54331892_A_G,2.928238e-05,-0.2657114,0.06358288,0.009652824,19,54412589,54447195,88,88,0.7696934,73.05146,19,54331892,G,1,0.2727273,0.79079983,ENSG00000105605
19_54679392_C_T,3.555071e-03,0.5805329,0.19914276,0.368087557,19,54754263,54761164,88,88,0.7764574,41.14800,19,54679392,T,1,0.3068182,0.62175963,ENSG00000105609
19_54665970_C_T,1.812138e-02,-0.3009787,0.12736450,0.800571372,19,54658899,54663620,88,88,0.6641172,61.89647,19,54665970,T,1,0.3522727,0.10114641,ENSG00000105617
19_54734845_G_T,9.301671e-03,0.2793480,0.10741002,0.594256362,19,54618837,54635140,88,88,0.6589594,60.81384,19,54734845,T,1,0.2272727,0.03595749,ENSG00000105618
19_54658102_C_G,2.565016e-03,-0.3452162,0.11447814,0.239061707,19,54610320,54619055,88,88,0.6574670,43.52220,19,54658102,G,1,0.4261364,0.82750674,ENSG00000105619


In [18]:
df0 = leads
df0 <- df0[-which(duplicated(df0$feature_id)),]
length(unique(df0$feature_id))
# df1 = df0[df0$feature_id %in% G_file$gene,]
df1 = df0[df0$feature_id %in% common_genes,]
length(unique(df1$feature_id))
# df1$BH_fdr = p.adjust(df1$empirical_feature_p_value, method = 'BH')
# length(unique(df1[df1$BH_fdr < 0.05,'feature_id']))
df1$q_value = qvalue(df1$empirical_feature_p_value)$qvalues
length(unique(df1[df1$q_value < 0.05,'feature_id']))

[1] 42791

[1] 12720

[1] 2603

In [19]:
max_qval = max(df1[df1$q_value < 0.1,'q_value'])
max_qval
emp_pv_star_matched = max(df1[df1$q_value==max_qval,"empirical_feature_p_value"])
emp_pv_star_matched

[1] 0.09989005

[1] 0.0548876

In [20]:
df3 = df1[df1$q_value < 0.05,]
df5 = inner_join(df3, res_all, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df5 <- df5[-which(duplicated(df5$feature_id)),]
nrow(df5)

nrow(df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),])
nrow(df5[df5$empirical_feature_p_value.bulk < emp_pv_star_all,])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 2590

[1] 2448

[1] 2448

In [14]:
leads_all = read.csv(paste0(bulk_folder,"top_qtl_results_all.txt"), sep = "\t")
nrow(leads_all)

[1] 49916

In [15]:
df0 = leads_all
df0 <- df0[-which(duplicated(df0$feature_id)),]
length(unique(df0$feature_id))
# df1 = df0[df0$feature_id %in% G_file$gene,]
df1 = df0[df0$feature_id %in% common_genes,]
length(unique(df1$feature_id))
df1$q_value = qvalue(df1$empirical_feature_p_value)$qvalues
length(unique(df1[df1$q_value < 0.05,'feature_id']))

[1] 49554

[1] 12720

[1] 9729

In [16]:
max_qval = max(df1[df1$q_value < 0.1,'q_value'])
max_qval
emp_pv_star_all = max(df1[df1$q_value==max_qval, "empirical_feature_p_value"])
emp_pv_star_all

[1] 0.09996513

[1] 0.4917055

In [19]:
##### Mean

In [26]:
mean_folder = "/hps/nobackup/hipsci/scratch/ComparingQtlMapping/SingleCell/PseudoBulk/mean/"

In [38]:
my_folder = paste0(mean_folder,"Run_Output_PCA20_88_log_TPM_scater_libsize_206/") # scran, 20PCs
# my_folder = paste0(mean_folder,"Run_Output_PCA20_88_log_TPM_scater_libsize_206_baynorm/") # baynorm, 20PCs

In [39]:
df0 = read.csv(paste0(my_folder,"top_qtl_results_all.txt"), sep = "\t")
df0 <- df0[-which(duplicated(df0$feature_id)),]

In [40]:
length(unique(df0$feature_id))
# df1 = df0[df0$feature_id %in% G_file$gene,]
df1 = df0[df0$feature_id %in% common_genes,]
length(unique(df1$feature_id))

df1$q_value = qvalue(df1$empirical_feature_p_value)$qvalues
length(unique(df1[df1$q_value < 0.05,'feature_id']))

df3 = df1[df1$q_value < 0.05,]
# nrow(df3)

[1] 50425

[1] 12720

[1] 1835

In [41]:
df4 = inner_join(df3, res, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df4 <- df4[-which(duplicated(df4$feature_id)),]
nrow(df4)

nrow(df4[(df4$empirical_feature_p_value.bulk < emp_pv_star_matched & 
         (df4$beta.sc*df4$beta.bulk)>0),])
nrow(df4[(df4$empirical_feature_p_value.bulk < emp_pv_star_matched),])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1835

[1] 889

[1] 904

In [42]:
df5 = inner_join(df3, res_all, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df5 <- df5[-which(duplicated(df5$feature_id)),]
nrow(df5)

nrow(df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),])
nrow(df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all),])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1829

[1] 1367

[1] 1471

In [43]:
dr_mean = df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),"feature_id"]

In [27]:
##### Total Mean

In [28]:
total_mean_folder = "/hps/nobackup/hipsci/scratch/ComparingQtlMapping/SingleCell/PseudoBulk/totmean/"

In [29]:
my_folder = paste0(total_mean_folder,"Run_Output_PCA20_88_log_TPM_scater_libsize_206/") # scran, 20PCs

In [30]:
df0 = read.csv(paste0(my_folder,"top_qtl_results_all.txt"), sep = "\t")
df0 <- df0[-which(duplicated(df0$feature_id)),]

In [31]:
length(unique(df0$feature_id))
# df1 = df0[df0$feature_id %in% G_file$gene,]
df1 = df0[df0$feature_id %in% common_genes,]
length(unique(df1$feature_id))
df1$q_value = qvalue(df1$empirical_feature_p_value)$qvalues
length(unique(df1[df1$q_value < 0.05,'feature_id']))

df3 = df1[df1$q_value < 0.05,]
# nrow(df3)

[1] 50429

[1] 12720

[1] 1305

In [32]:
df4 = inner_join(df3, res, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df4 <- df4[-which(duplicated(df4$feature_id)),]
nrow(df4)

nrow(df4[(df4$empirical_feature_p_value.bulk < emp_pv_star_matched & 
         (df4$beta.sc*df4$beta.bulk)>0),])
nrow(df4[(df4$empirical_feature_p_value.bulk < emp_pv_star_matched),])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1305

[1] 768

[1] 781

In [33]:
df5 = inner_join(df3, res_all, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df5 <- df5[-which(duplicated(df5$feature_id)),]
nrow(df5)

nrow(df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),])
nrow(df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all),])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1301

[1] 1046

[1] 1119

In [34]:
d_mean = df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),"feature_id"]

In [35]:
##### Median

In [36]:
median_folder = "/hps/nobackup/hipsci/scratch/ComparingQtlMapping/SingleCell/PseudoBulk/median/"

In [37]:
my_folder = paste0(median_folder,"Run_Output_PCA20_88_log_TPM_scater_libsize_206/") # scran, 20PCs

In [38]:
# df = read.csv(paste0(my_folder,"qtl_results_all.txt"), sep = "\t")
# df <- df[order(df$empirical_feature_p_value, df$p_value ),]
# df0 <- df[-which(duplicated(df$feature_id)),]
# length(unique(df0$feature_id))

In [39]:
df0 = read.csv(paste0(my_folder,"top_qtl_results_all.txt"), sep = "\t")
median_genes = unique(df0$feature_id)
length(median_genes)

[1] 13628

In [40]:
length(unique(df0$feature_id))
# df1 = df0[df0$feature_id %in% G_file$gene,]
df1 = df0[df0$feature_id %in% common_genes,]
length(unique(df1$feature_id))
df1$q_value = qvalue(df1$empirical_feature_p_value)$qvalues
length(unique(df1[df1$q_value < 0.05,'feature_id']))

df3 = df1[df1$q_value < 0.05,]
nrow(df3)

df4 = inner_join(df3, res, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df4 <- df4[-which(duplicated(df4$feature_id)),]
nrow(df4)

nrow(df4[(df4$empirical_feature_p_value.bulk < emp_pv_star_matched & 
         (df4$beta.sc*df4$beta.bulk)>0),])
nrow(df4[(df4$empirical_feature_p_value.bulk < emp_pv_star_matched),])

df5 = inner_join(df3, res_all, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df5 <- df5[-which(duplicated(df5$feature_id)),]
nrow(df5)

nrow(df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),])
nrow(df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all),])

[1] 13628

[1] 12720

[1] 1337

[1] 1340

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

...

[1] 668

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1333

[1] 952

[1] 1034

In [41]:
dr_median = df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),"feature_id"]

In [42]:
###### Total Median

In [32]:
totmedian_folder = "/hps/nobackup/hipsci/scratch/ComparingQtlMapping/SingleCell/PseudoBulk/totmedian/"

In [33]:
my_folder = paste0(totmedian_folder,"Run_Output_PCA20_88_log_TPM_scater_libsize_206/") # scran, 20PCs

In [34]:
df0 = read.csv(paste0(my_folder,"top_qtl_results_all.txt"), sep = "\t")
df0 <- df0[-which(duplicated(df0$feature_id)),]

In [35]:
length(unique(df0$feature_id))
# df1 = df0[df0$feature_id %in% G_file$gene,]
df1 = df0[df0$feature_id %in% common_genes,]
length(unique(df1$feature_id))
df1$q_value = qvalue(df1$empirical_feature_p_value)$qvalues
length(unique(df1[df1$q_value < 0.05,'feature_id']))

df3 = df1[df1$q_value < 0.05,]
nrow(df3)

[1] 12727

[1] 12720

[1] 776

[1] 776

In [36]:
df4 = inner_join(df3, res, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df4 <- df4[-which(duplicated(df4$feature_id)),]
nrow(df4)

nrow(df4[(df4$empirical_feature_p_value.bulk < emp_pv_star_matched & 
         (df4$beta.sc*df4$beta.bulk)>0),])
nrow(df4[(df4$empirical_feature_p_value.bulk < emp_pv_star_matched),])

df5 = inner_join(df3, res_all, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df5 <- df5[-which(duplicated(df5$feature_id)),]
nrow(df5)

nrow(df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),])
nrow(df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all),])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 774

[1] 470

[1] 483

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 774

[1] 625

[1] 681

In [37]:
d_median = df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),"feature_id"]

In [49]:
##### Sum

In [50]:
sum_folder = "/hps/nobackup/hipsci/scratch/ComparingQtlMapping/SingleCell/PseudoBulk/sum/"

In [51]:
my_folder = paste0(sum_folder,"Run_Output_PCA20_88_log_TPM_edgeR_libsize_206/") # edgeR, 20PCs

In [52]:
df0 = read.csv(paste0(my_folder,"top_qtl_results_all.txt"), sep = "\t")
df0 <- df0[-which(duplicated(df0$feature_id)),]

In [53]:
length(unique(df0$feature_id))
# df1 = df0[df0$feature_id %in% G_file$gene,]
df1 = df0[df0$feature_id %in% common_genes,]
length(unique(df1$feature_id))
df1$q_value = qvalue(df1$empirical_feature_p_value)$qvalues
length(unique(df1[df1$q_value < 0.05,'feature_id']))

df3 = df1[df1$q_value < 0.05,]
nrow(df3)

[1] 50427

[1] 12720

[1] 1463

[1] 1463

In [54]:
df4 = inner_join(df3, res, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df4 <- df4[-which(duplicated(df4$feature_id)),]
nrow(df4)

nrow(df4[(df4$empirical_feature_p_value.bulk < emp_pv_star_matched & 
         (df4$beta.sc*df4$beta.bulk)>0),])
nrow(df4[(df4$empirical_feature_p_value.bulk < emp_pv_star_matched),])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1451

[1] 819

[1] 832

In [55]:
df5 = inner_join(df3, res_all, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df5 <- df5[-which(duplicated(df5$feature_id)),]
nrow(df5)

nrow(df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),])
nrow(df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all),])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1448

[1] 1153

[1] 1232

In [56]:
dr_sum = df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),"feature_id"]

In [57]:
##### Total Sum

In [58]:
total_sum_folder = "/hps/nobackup/hipsci/scratch/ComparingQtlMapping/SingleCell/PseudoBulk/totsum/"

In [59]:
my_folder = paste0(total_sum_folder,"Run_Output_PCA20_88_log_TPM_edgeR_libsize_206/") # edgeR, 20PCs

In [60]:
# df = read.csv(paste0(my_folder,"qtl_results_all.txt"), sep = "\t")
# df <- df[order(df$empirical_feature_p_value, df$p_value ),]
# df0 <- df[-which(duplicated(df$feature_id)),]
# # nrow(df0)
# length(unique(df0$feature_id))

In [61]:
# write.table(df0, "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_eqtl/top_qtl_results_totsum.txt")

In [62]:
df0 = read.csv("/hps/nobackup/stegle/users/acuomo/all_scripts/sc_eqtl/top_qtl_results_totsum.txt", sep = ' ')
head(df0,2)

,feature_id,snp_id,p_value,beta,beta_se,empirical_feature_p_value,feature_chromosome,feature_start,feature_end,n_samples,n_e_samples,alpha_param,beta_param,snp_chromosome,snp_position,assessed_allele,call_rate,maf,hwe_p
1955841,ENSG00000203875,6_86387888_G_T,1.514293e-27,1.010238,0.09289432,6.375394e-24,6,86370710,86388451,87,87,0.7886659,17.12808,6,86387888,T,1,0.4655172,0.6673797
6927785,ENSG00000204650,17_43679121_G_A,9.296484e-23,-1.201981,0.12240973,2.026242e-20,17,43697694,43725582,87,87,0.7567724,29.08926,17,43679121,A,1,0.2701149,0.1020851


In [63]:
length(unique(df0$feature_id))
# df1 = df0[df0$feature_id %in% G_file$gene,]
df1 = df0[df0$feature_id %in% common_genes,]
length(unique(df1$feature_id))
df1$q_value = qvalue(df1$empirical_feature_p_value)$qvalues
length(unique(df1[df1$q_value < 0.05,'feature_id']))

df3 = df1[df1$q_value < 0.05,]
nrow(df3)

df4 = inner_join(df3, res, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df4 <- df4[-which(duplicated(df4$feature_id)),]
nrow(df4)

nrow(df4[(df4$empirical_feature_p_value.bulk < emp_pv_star_matched & 
         (df4$beta.sc*df4$beta.bulk)>0),])
nrow(df4[(df4$empirical_feature_p_value.bulk < emp_pv_star_matched),])

df5 = inner_join(df3, res_all, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df5 <- df5[-which(duplicated(df5$feature_id)),]
nrow(df5)

nrow(df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),])
nrow(df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all),])

[1] 50429

[1] 12720

[1] 1174

[1] 1174

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

...

[1] 722

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1170

[1] 951

[1] 1024

In [64]:
d_sum = df5[(df5$empirical_feature_p_value.bulk < emp_pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),"feature_id"]

In [70]:
# length(dr_mean)
# length(dr_mean[dr_mean %in% dr_mean])
# length(dr_median)
length(dr_median[dr_median %in% dr_mean])
# length(dr_sum)
length(dr_sum[dr_sum %in% dr_mean])
#
# length(d_mean)
length(d_mean[d_mean %in% dr_mean])
# length(d_median)
length(d_median[d_median %in% dr_mean])
# length(d_sum)
length(d_sum[d_sum %in% dr_mean])

[1] 797

[1] 933

[1] 932

[1] 443

[1] 806

In [44]:
length(d_median[d_median %in% dr_mean])

[1] 567